In [4]:
import tensorflow as tf
import numpy as np
import data as d
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
d.recompute()

[45927 67343 11656   995    52]
[  1   1   3  26 485]
[ 1.          1.          1.00413223  1.05165289  2.        ]
[45927 67343 11656   995    52]

/home/nilabhra/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/nilabhra/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)



[  1   1   3  26 485]
[ 1.          1.          1.00413223  1.05165289  2.        ]


In [5]:
def precision(predictions, labels):
    p = [np.argmax(x) for x in predictions]
    labs = [np.argmax(x) for x in labels]
    return 100 * precision_score(labs, p, average='macro')
def confusion(predictions, labels):
    p = [np.argmax(x) for x in predictions]
    labs = [np.argmax(x) for x in labels]
    return confusion_matrix(labs, p)

In [38]:
graph = tf.Graph()
with graph.as_default():
    learning_rate = 0.001
    display_step = 1
    bacthes_per_epoch = 10000
    # Network Parameters
    n_input = 50
    n_hidden_1 = 50
    n_hidden_2 = 50
    n_hidden_3 = 50
    n_hidden_4 = 50
    n_classes = 5
    x = tf.placeholder(tf.float32, [None, n_input])
    y = tf.placeholder(tf.float32, [None, n_classes])
    c_w = tf.placeholder(tf.float32, [200])
#     tf_valid_dataset = tf.constant(valid_dataset)
    dropout_rate = tf.placeholder(tf.float32)
    
    def multilayer_perceptron(X, weights, biases):
        layer_1 = tf.nn.relu(
            tf.add(
                tf.matmul(X, weights['h1']),
                biases['b1'])
            )
        layer_2 = tf.nn.relu(
            tf.add(
                tf.matmul(layer_1, weights['h2']),
                biases['b2'])
            )
        layer_3 = tf.nn.relu(
            tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
            )
        layer_4 = tf.nn.relu(
            tf.add(
                tf.matmul(layer_3, weights['h4']),
                biases['b4'])
            )
        return tf.add(
            tf.matmul(tf.nn.dropout(layer_4, dropout_rate), weights['out']),
            biases['out']
            )
    
    
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
        'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
        'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'b3': tf.Variable(tf.truncated_normal([n_hidden_3])),
        'b4': tf.Variable(tf.truncated_normal([n_hidden_4])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

    # Construct model
    pred = multilayer_perceptron(x, weights, biases)
    prediction = tf.nn.softmax(pred)
    # Define loss and optimizer
    total_l2_loss = 0
#     for k, v in weights.items():
#         total_l2_loss += tf.nn.l2_loss(v)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y) * c_w) #+ 0.0001 * (total_l2_loss)
#     cost = precision(prediction, y)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.initialize_all_variables()
#     valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset))

In [39]:
prev_loss = 100000
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    sess.run(init)
    saver = tf.train.Saver()
    for i in range(bacthes_per_epoch):
        batch_xs, batch_ys, cw = d.train_batch_data(40)
        train_feed_dict = {
            x: batch_xs,
            y: batch_ys,
            dropout_rate: 0.5,
            c_w: cw
        }

        if i % 500 == 0:
            test_feed_dict = {
                x: batch_xs,
                y: batch_ys,
                dropout_rate: 1.0,
                c_w: cw
            }
            _, c = sess.run(
                [optimizer, cost], feed_dict=train_feed_dict
            )
#                 if abs(prev_loss - c) < 0.0001:
#                     break
#                 prev_loss = c
            print "Minibatch loss at step", i, ":", c
            batch_precision = precision(prediction.eval(feed_dict=test_feed_dict),batch_ys)
            print "Minibatch precision at step", i, ":", batch_precision
#                 validation_precision = precision(
#                     valid_prediction.eval(feed_dict=test_feed_dict),
#                     valid_labels
#                 )
#                 print(
#                     "Validation precission at step", step, ":",
#                     validation_precision,
#                     end='\n',
#                     flush=True
#                 )
        else:
            _, c = sess.run([optimizer, cost], feed_dict=train_feed_dict)
#                 if abs(prev_loss - c) < 0.0001:
#                     break
#                 prev_loss = c


    #Test model on test dataset
    
    tdata = np.load('out/new_input.npz')['test']
    a, b = tdata[:, :-1], tdata[:, -1]
    b = LabelBinarizer().fit_transform(b-1)
    preds = prediction.eval(feed_dict={x:a,y:b,dropout_rate:1.0})
    print("Test Precision:", precision(preds, b))
    print(confusion(preds, b))  

Minibatch loss at step 0 : 16045.5
Minibatch precision at step 0 : 4.08163265306
Minibatch loss at step 500 : 249.336
Minibatch precision at step 500 : 89.592760181
Minibatch loss at step 1000 : 66.209
Minibatch precision at step 1000 : 84.0537240537
Minibatch loss at step 1500 : 9.37548
Minibatch precision at step 1500 : 84.1153255347
Minibatch loss at step 2000 : 2.58868
Minibatch precision at step 2000 : 87.069124424
Minibatch loss at step 2500 : 1.98713
Minibatch precision at step 2500 : 83.1708005646
Minibatch loss at step 3000 : 1.95953
Minibatch precision at step 3000 : 84.5224853011
Minibatch loss at step 3500 : 1.98759
Minibatch precision at step 3500 : 86.736655281
Minibatch loss at step 4000 : 1.82903
Minibatch precision at step 4000 : 88.191681736
Minibatch loss at step 4500 : 1.62275
Minibatch precision at step 4500 : 89.3103448276
Minibatch loss at step 5000 : 1.88339
Minibatch precision at step 5000 : 86.3111467522
Minibatch loss at step 5500 : 1.75857
Minibatch precisio